# Домашнее задание по теме "Рекомендации на сонове содержания"**

1. Использовать dataset MovieLens
1. Построить рекомендации (регрессия, предсказываем оценку) на фичах:
1. TF-IDF на тегах и жанрах
1. Средние оценки (+ median, variance, etc.) пользователя и фильма
1. Оценить RMSE на тестовой выборке

In [1]:
import pandas as pd
import numpy as np

from recommendation_models import ContentBasedMovieRecommendations

Для домашней работы был реализован класс ContentBasedMovieRecommendations. 

Общий алгоритм работы:

- подаем на вход пути к файлам movies.csv, ratings.csv и tags.csv, работа класса строго привязана к формату датасета MovieLens;
- указываем идентификатор пользователя из датасета MovieLens, для которого будем строить модель;
- с помощью метода fit обучаем модель;
- с помощью метода evaluate расчитываем MAE и RMSE регрессии предсказания оценок пользователя;
- с помощью метода get_recommendations находим топ фильмов для рекомендации пользователю на основании идентификатора просмотренного фильма.

### Подробный алгоритм работы модели рекомендаций

#### 1.  Подготовка датафреймов для преобразования

Для каждого фильма находится средний и медианный рейтинг всех пользователей, а так же стандартное отклонение общего рейтинга. Если у фильма есть средняя оценка, но нет данных по стандартному отклонению, то такие пропуски в СКО заполняются нулями. 

Фильмы без оценок пользователей удаляются, так как будут проблемы при получении оценки с помощью регрессии.

Для каждого фильма собираются в отдельном столбце все тэги пользователя выбранного пользователя. Если пользователь не оставил ни одного тега фильму, то такой пропуск заполняется значением "NO_USER_TAG". Еще обрабатывается столбец с жанром для каждого фильма в базе для возможности передать данные в модель TF-IDF.

#### 2.  Обучение модели

На первом этапе мы обучаем модель NearestNeighbors. Обучаем ее только на текстовых данных в виде TF-ШВА без средних оценок. Если искать соседей в том числе по средним оценкам, то при просмотре пользователем фильма со средним рейтингом 3, подбираться будут тоже аналогичные фильмы с такой же плохой средней оценкой. При этом далеко не факт, что сам пользователь поставит оценку лучше, а не близкую к средней. Для обучения совмещаем данные из тегов и жанров в одно тектовое поле и получаем матрицу TF-IDF, масштабируем данные и обучаем модель NearestNeighbors на всей базе фильмов, кроме тех, которые пользователь уже посмотрел. То есть у них нет оценки пользователя.

На втором этапе нам нужно обучить модель регрессии, которая будет предсказывать предположительную оценку выбранного пользователя рекомендуемому фильму. В качестве регрессора выбран SVR. Для более качественного обучения модели нам требуется выбирать пользователей, которые поставили много оценок. Для обучения берем не масштабированные данные TF-IDF с предыдущего шага. Добавляем к матрице средние, медианные оценки и СКО. А так же целевую переменную в виде оценки пользователя к каждому фильму. Удаляем из матрицы фильмы, которым выбранный пользователь не поставил оценку. Далее, разделяем выборку на тестовую и обучающую, проводим стандартизацию датасетов и обучаем нашу модель регрессии. Качество обучения регрессии можно проверить через метод класса evaluate.

#### 3.  Построение рекомендаций

Для построения рекомендаций используется метод get_recommendations, в котором в качестве параметра передаем идентификатор просмотренного пользователем фильма. 

Общая идея построения рекомендаций следующая:

1. находим большое количество похожих фильмов по тегам и жанрам, например, 100 фильмов;
1. добавляем к этой выдаче определенное количество случайных фильмов с высоким средним рейтингом, чтобы пользователь мог попробовать посмотреть немного другие хорошие фильмы;
1. для полученной выборки предсказываем оценку пользователя и выбираем топ фильмов с максимальной предсказанной оценкой.
1. в итоговый топ может как попасть случайный фильм, так и не попасть. Это зависит от предсказанной оценки пользователя и размера топовой выборки.

In [2]:
movies_csv = 'movies.csv'
ratings_csv = 'ratings.csv'
tags_csv = 'tags.csv'

"""
Параметры
---------
user_id : int или str
  Идентификатор пользователя, для которого будем строить модель. Принимает значение int или 'auto'. D случае 
  auto пользователь будет выбран автоматически по критерию максимального количества оценок к фильмам
movies_csv : str
  Путь к файлу movies.csv из датасета MovieLens
ratings_csv : str
  Путь к файлу ratings.csv из датасета MovieLens
tags_csv : str
  Путь к файлу tags.csv из датасета MovieLens
svr_C : float
  Коэффициент регуляризации для регрессора SVR, смотри справку по sklearn.svm.SVR
svr_tol : float
   Минимальный допуск изменения качества SVR до остановки обучения модели, смотри справку по sklearn.svm.SVR
svr_kernel : str
  Тип ядра для регрессора SVR, смотри справку по sklearn.svm.SVR
svr_deegre : str
  Степень полинома для ядра poly, смотри справку по sklearn.svm.SVR
recommend_count : int
  Количество фильмов для итоговой рекомендации
random_recommend_films : int
  Количество случайных фильмов с высоким общим рейтингом, которые будут подмешиваться в рекомендации
n_neighbors : int
  Количество сходных по жанру и тегам фильмов для выдачи рекомендаций. При получении рекомендации
  модель выберет указанное количество фильмов, предскажет для них оценку пользователя и выберет топ фильмов 
  уровню оценки. размер топа определяется в recommend_count.
"""
model = ContentBasedMovieRecommendations(user_id='auto', movies_csv=movies_csv, 
                                         ratings_csv=ratings_csv, tags_csv=tags_csv,
                                         svr_C=0.5, svr_tol=1e-7, recommend_count=10, random_recommend_films=5)

In [3]:
# Обучаем модель. Можно пробовать менять параметры модели SVR через свойства класса, 
# при этом подготовка датафрейма для обучения из этапа номер 1 будет проходить только один раз для экономии времени.
model.svr_C = 0.1
model.svr_tol = 1e-7
"""
Параметры
---------
test_size : float
  Размер тестовой выборки для обучения регрессионной модели предсказания оценки пользователя. 
  Принимает значения от 0 до 1
"""
model.fit(test_size=0.2)

# Проверяем статистику модели после обучения
model.print_stat()

print('='*50)
# Проверяем качество предсказаний оценки пользователя на регрессионной модели
print('Качество работы регрессионной модели предсказания оценок:\n')
model.evaluate()

Идентификатор пользователя: 414
Кол-во оценок пользователя к фильмам: 2698
Всего кол-во фильмов: 9724
Кол-во фичей при обучении регрессионной модели: 24
Кол-во фильмов при обучении регрессионной модели: 2158
Качество работы регрессионной модели предсказания оценок:

MAE: 0.5029865636399479
RMSE: 0.6821888532086684


Как видно из MAE мы можем в среднем ошибаться на половину балла.

Найдем рекомендуемые фильмы для пользователя с выбранным идентификатором. 
Предположим, что пользователь посмотрел фильм с идентификатором 107 и ему нужно выдать рекомендации по этому фильму:

In [4]:
model.get_recommendations(movie_id=107)

Пользователь посмотрел следующий фильм:


,movieId,title,genres,rating_mean,rating_median,rating_std,user_rating,user_tags
95,107,Muppet Treasure Island (1996),adventure children comedy musical,3.326923,3.0,0.92674,3.0,NO_USER_TAG


ТОП-10 рекомендованных фильмов для просмотра в порядке максимальной предсказанной оценки пользователя:


,movieId,title,genres,rating_mean,rating_median,rating_std,user_tags,distance,user_rating_predict
0,506,Orlando (1992),drama fantasy romance,4.500000,4.50,0.707107,NO_USER_TAG,None,4.246555
1,1066,Shall We Dance (1937),comedy musical romance,4.500000,4.50,0.577350,NO_USER_TAG,4.612685,4.139664
2,73854,"Rudolph, the Red-Nosed Reindeer (1964)",adventure animation children fantasy musical,3.833333,4.00,0.763763,NO_USER_TAG,4.117975,4.125484
3,6345,"Chorus Line, A (1985)",comedy drama musical,4.666667,5.00,0.577350,NO_USER_TAG,4.480666,4.097976
4,1566,Hercules (1997),adventure animation children comedy musical,3.451613,4.00,1.035655,NO_USER_TAG,3.203599,4.027364
5,44889,Reefer Madness: The Movie Musical (2005),comedy drama musical,4.750000,4.75,0.353553,NO_USER_TAG,4.480666,4.009247
6,8580,Into the Woods (1991),adventure comedy fantasy musical,5.000000,5.00,0.000000,NO_USER_TAG,4.553087,3.971669
7,945,Top Hat (1935),comedy musical romance,4.071429,4.00,0.838082,NO_USER_TAG,4.612685,3.970729
8,104875,"History of Future Folk, The (2012)",adventure comedy musical scifi,4.000000,4.00,0.707107,NO_USER_TAG,4.21303,3.969308
9,7888,How to Succeed in Business Without Really Tryi...,comedy musical,4.250000,4.25,1.060660,NO_USER_TAG,4.500174,3.960656
